In [3]:
from mongoengine import connect
import math
import json
import sys
sys.path.insert(0, '..\\mongo-db-scripts')
from Bug import Bug

connect('bug_reports_db', host='127.0.0.1', port=27017)

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [4]:
def count_changes(history):
  counted_changes = {
    'cf': 0,
    'not_cf': 0
  }

  for changes in history:
    for change in changes['changes']:
      field_name = change['field_name']

      if (field_name.startswith('cf_')):
        counted_changes['cf'] += 1
      else:
        counted_changes['not_cf'] += 1
      
  return counted_changes

In [ ]:
BUGS_LIMIT = 50000
total_bugs = Bug.objects.count()
bugs_per_iteration = math.ceil(total_bugs/BUGS_LIMIT)

bugs_counting_changes = []

for i in range(bugs_per_iteration):
  skip_quantity = i * BUGS_LIMIT
  result = Bug.objects.only('bug_id', 'history').limit(BUGS_LIMIT).skip(skip_quantity)

  bugs = [bug.to_mongo().to_dict() for bug in result]

  for bug in bugs:
    temp_bug = count_changes(bug['history'])
    temp_bug['bug_id'] = bug['bug_id']
    bugs_counting_changes.append(temp_bug)
                    

In [7]:
with open('data\\cf_notcf_counting_final.json', 'w') as outfile:
  json.dump(bugs_counting_changes, outfile) 